In [14]:
import dspy
import os
from dotenv import load_dotenv
load_dotenv()
load_dotenv('.secrets')

True

In [15]:
class BoardGameResearchPlanner(dspy.Signature):
    """Takes the user query and plans deep research by figuring out subquestions that relate to the theme the user is interested in knowing more"""
    question = dspy.InputField(desc="The user's initial query when researching board games")
    subquestions: list[str] = dspy.OutputField(desc="A list of 5 or more subquestions that dive deeper in the theme the user is researching.")

class BoardGameResearcher(dspy.Signature):
    """Takes the original question and the related question and finds an answer to the subquestion that is related to the original question"""
    question = dspy.InputField(desc="The user's initial query about board games")
    subquestion: str = dspy.InputField(desc="The subquestion you are doing deep research on")
    context = dspy.OutputField(desc="The context learned by answering the subquestion while relevant to the initial question")

class BoardGameResearchSummarizer(dspy.Signature):
    """Condense the retrieved information provided in the context into a focused summary that answers the original question."""
    question = dspy.InputField(desc="The user's initial query about board games")
    context: list[str] = dspy.InputField(desc="Research context discovered about the board games you are about to recommend")
    summary = dspy.OutputField(desc="Condense retrieved information into a focused summary relevant to the provided context and user question. Provide specific details supporting the answer to the question.")


class BoardGameAssistant(dspy.Module):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.plan_research = dspy.ChainOfThought(BoardGameResearchPlanner)
        self.research = dspy.ChainOfThought(BoardGameResearcher)
        self.summarize = dspy.ChainOfThought(BoardGameResearchSummarizer)

    def forward(self, question):
        subquestions = self.plan_research(question=question).subquestions
        research_context = []
        print(f"Question: {question} research tasks: {len(subquestions)}")
        for subquestion in subquestions:
            print(f" - Researching: {subquestion}")
            research = self.research(subquestion=subquestion, question=question)
            research_context = research_context + [research]

        final_research = self.summarize(context=list(map(lambda r: r.context, research_context)), question=question)

        return dspy.Prediction(answer=final_research.summary, research=research_context)

In [16]:
lm = dspy.LM('openai/gpt-4o', api_key=os.getenv("OPENAI_API_KEY"))
dspy.configure(lm=lm)

In [17]:
assistant = BoardGameAssistant()
recommendation = assistant(question="What is the best opening move in Catan?")

Question: What is the best opening move in Catan? research tasks: 7
 - Researching: What are the most important resources to prioritize in the opening move of Catan?
 - Researching: How does the number of players affect the strategy for the opening move in Catan?
 - Researching: What are the advantages and disadvantages of starting near ports in Catan?
 - Researching: How should one balance between resource diversity and production frequency in the opening move?
 - Researching: What are common mistakes to avoid when choosing the initial settlement locations in Catan?
 - Researching: How can the initial placement strategy change based on the specific layout of the board?
 - Researching: What role does trading play in determining the best opening move in Catan?


In [20]:
print(f"Answer: {recommendation.answer}")

Answer: The best opening move in Catan involves placing initial settlements to maximize access to key resources like brick, wood, and wheat, ensuring a balanced mix and high-frequency numbers for steady resource flow. Strategy should adapt based on the number of players, with a focus on resource balance in 4-player games. Consideration of ports for trading advantages, while balancing resource diversity, is crucial. Understanding the board layout and potential trading opportunities enhances strategic positioning, avoiding common mistakes like resource imbalance and poor expansion potential.


In [21]:
print(f"Research: {recommendation.research}")

Research: [Prediction(
    reasoning='In Catan, the opening move is crucial because it sets the foundation for your resource production throughout the game. The most important resources to prioritize in the opening move are typically brick and wood, as they are essential for building roads and settlements, which are key to expanding your territory and accessing more resources. Additionally, wheat is important for building settlements and cities, while ore becomes more valuable as the game progresses for upgrading settlements to cities. Balancing these resources based on the board layout and your strategy is essential for a strong start.',
    context='The best opening move in Catan involves strategically placing your initial settlements to maximize access to key resources like brick, wood, and wheat. These resources are crucial for early game expansion and development. By prioritizing these resources, players can build roads and settlements more efficiently, which is vital for gaining 

In [19]:
lm.inspect_history(n=10)





[2025-02-03T12:23:56.339532]

System message:

Your input fields are:
1. `question` (str): The user's initial query when researching board games

Your output fields are:
1. `reasoning` (str)
2. `subquestions` (list[str]): A list of 5 or more subquestions that dive deeper in the theme the user is researching.

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## subquestions ## ]]
{subquestions}        # note: the value you produce must be pareseable according to the following JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `subquestions`.


User message:

[[ ## question ## ]]
What is the best opening move in Catan?

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## sub

In [22]:
monopoly = BoardGameAssistant()
monoreco = assistant(question="What is the best opening move in Monopoly?")

Question: What is the best opening move in Monopoly? research tasks: 8
 - Researching: What are the most statistically likely properties to land on in the first few turns of Monopoly?
 - Researching: How does the number of players in a game of Monopoly affect the best opening strategy?
 - Researching: What are the advantages and disadvantages of buying railroads early in the game?
 - Researching: Which properties are considered the most valuable in terms of return on investment?
 - Researching: How do house rules or variations of Monopoly impact the effectiveness of different opening moves?
 - Researching: What role does trading play in establishing a strong position early in the game?
 - Researching: How important is it to acquire a complete color set early in the game?
 - Researching: What are some common mistakes players make in the opening moves of Monopoly?


In [23]:
print(monoreco.answer)

The best opening move in Monopoly involves purchasing properties that are statistically more likely to be landed on early, such as "Oriental Avenue," "Vermont Avenue," and "Connecticut Avenue." Prioritizing properties in the orange and red groups is also strategic due to their high return on investment. Balancing the acquisition of railroads for steady income with the goal of completing color sets is crucial. Additionally, being open to strategic trades and considering house rules or variations can enhance one's position. This approach combines immediate gains with long-term strategic advantages.


In [24]:
print(monoreco.research)

[Prediction(
    reasoning='In the early stages of Monopoly, the properties that players are most likely to land on are those that are closest to the starting point, "Go." This is because players begin the game by rolling two six-sided dice, which means the most common roll is a 7. Therefore, properties that are 6, 7, or 8 spaces away from "Go" are statistically more likely to be landed on in the first few turns. These properties include "Oriental Avenue," "Vermont Avenue," and "Connecticut Avenue." Additionally, the "Chance" and "Community Chest" cards can also influence early game movement, but the initial dice rolls are the primary factor in determining which properties are landed on first.',
    context='Understanding which properties are statistically more likely to be landed on in the first few turns of Monopoly can inform the best opening move. Since properties like "Oriental Avenue," "Vermont Avenue," and "Connecticut Avenue" are more likely to be landed on early, prioritizing 